## Import

In [0]:
#pip install tensorflow
#pip install plotly
#pip install bs4
#pip install keras
#pip install lxml

In [0]:
import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import re
from io import StringIO
from datetime import datetime, timedelta
import requests
import math
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots  
from plotly.offline import iplot
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import json
import csv
from bs4 import BeautifulSoup
import lxml
from lxml import html
from io import StringIO
import pyspark
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import round

## Données historiques

On va webscrapper les données historiques de plusieurs valeurs technologiques, il s'agit de 'Apple Inc - AAPL', 'Tesla Inc - TSLA', 'Microsoft Corporation - MSFT', 
                                                                                      'Amazon.com Inc - AMZN', 'Alphabet Inc - GOOGL',
                                                                                  'Meta Platforms Inc - FB' et 'NVIDIA Corporation - NVDA'

In [0]:
class WebScrapStock:
    
    timeout = 30
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    stock_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


    def __init__(self, stock_name, days_back=7):
        self.stock_name = stock_name
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)

    def get_crumb(self):
        response = self.session.get(self.crumb_link.format(self.stock_name), timeout=self.timeout, verify = False, headers=self.headers_)
        response.raise_for_status()
        match = re.search(self.crumble_regex, response.text)
        
        if not match:
            raise ValueError("Pas de réponse de Yahoo Finance")
        else:
            self.crumb = match.group(1)

    def get_stock(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.stock_link.format(quote=self.stock_name, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url, verify = False, headers=self.headers_)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text), parse_dates=['Date'])
        df["Stock"] = self.stock_name
        df = df[["Date", "Stock", "Open", "High", "Low", "Close", "Adj Close", "Volume"	]]
        return df, url

In [0]:

#On récupère les données historiques des différentes actions pour 10 années
jours = 360*10
df1, url1 = WebScrapStock(stock_name  = 'AAPL', days_back=jours).get_stock()
df2, url2 = WebScrapStock(stock_name  = 'TSLA', days_back=jours).get_stock()
df3, url3 = WebScrapStock(stock_name  = 'MSFT', days_back=jours).get_stock()
df4, url4 = WebScrapStock(stock_name  = 'AMZN', days_back=jours).get_stock()
df5, url5 = WebScrapStock(stock_name  = 'GOOGL', days_back=jours).get_stock()
df6, url6 = WebScrapStock(stock_name  = 'FB', days_back=jours).get_stock()
df7, url7 = WebScrapStock(stock_name  = 'NVDA', days_back=jours).get_stock()

df8 = df1.append(df2)
df9 = df8.append(df3)
df10 = df9.append(df4)
df11 = df10.append(df5)
df12 = df11.append(df6)
df13 = df11.append(df7)


data_ = df13.copy()
data_

,Date,Stock,Open,High,Low,Close,Adj Close,Volume
0,2012-03-05,AAPL,19.479286,19.552856,18.785713,19.041430,16.325459,809124400
1,2012-03-06,AAPL,18.702143,19.060356,18.436428,18.937857,16.236658,810238800
2,2012-03-07,AAPL,19.171429,19.206429,18.689285,18.953215,16.249825,798520800
3,2012-03-08,AAPL,19.096071,19.392500,19.004286,19.356787,16.595827,516457200
4,2012-03-09,AAPL,19.436071,19.562143,19.396786,19.470358,16.693203,418919200
...,...,...,...,...,...,...,...,...
2477,2022-01-05,NVDA,289.489990,294.160004,275.329987,276.040009,276.040009,49806400
2478,2022-01-06,NVDA,276.399994,284.380005,270.649994,281.779999,281.779999,45418600
2479,2022-01-07,NVDA,281.410004,284.220001,270.570007,272.470001,272.470001,40920100
2480,2022-01-10,NVDA,265.809998,274.690002,256.440002,274.000000,274.000000,59468100


In [0]:

fig = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(
x = df2["Date"],
y = df2["Close"],
mode = "lines",
name = "AAPL ",
marker = dict(color = 'grey'))

trace2 = go.Scatter(
x = df2["Date"],
y = df2["Close"],
mode = "lines",
name = "TSLA",
marker = dict(color = 'black'))

trace3 = go.Scatter(
x = df3["Date"],
y = df3["Close"],
mode = "lines",
name = "MSFT ",
marker = dict(color = 'red'))


trace4 = go.Scatter(
x = df4["Date"],
y = df4["Close"],
mode = "lines",
name = "AMZN",
marker = dict(color = 'green'))

trace5 = go.Scatter(
x = df5["Date"],
y = df5["Close"],
mode = "lines",
name = "GOOGL",
marker = dict(color = 'yellow'))


trace6 = go.Scatter(
x = df6["Date"],
y = df6["Close"],
mode = "lines",
name = "FB",
marker = dict(color = 'blue'))


trace7 = go.Scatter(
x = df7["Date"],
y = df7["Close"],
mode = "lines",
name = "NVDA",
marker = dict(color = 'pink'))


fig.add_trace(trace1, secondary_y=True)
fig.add_trace(trace2, secondary_y=True)
fig.add_trace(trace3, secondary_y=True)
fig.add_trace(trace4, secondary_y=True)
fig.add_trace(trace5, secondary_y=True)
fig.add_trace(trace6, secondary_y=True)
fig.add_trace(trace7, secondary_y=True)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='Red', rangeslider_visible=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black');
data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]

layout = dict(autosize=False,
                  width=1000,
                  height=550,
                  title = "Évolution du cours de l'action depuis " + str(data_.iloc[0, 0])[:10],
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de fermetures",ticklen= 5, zeroline= True, showline=True, showgrid=False))
fig = dict(data = data, layout = layout)
#fig.update_xaxes()
iplot(fig)

On voit que le titre AMZN a un coût plus élevé que les autres. On remarque aussi une baisse de tous les cours vers 2020 après une ralentissment des marchés financiers. Ensuite s'en suit un décollage, coincidant avec la hausse des valeurs technologiques.

## Data engineering avec Spark

In [0]:
df_spark=spark.createDataFrame(data_)
print("La base de données contient", df_spark.count(), "observations et", len(df_spark.columns), "variables.")

df_spark = df_spark.orderBy(['Stock', 'Date'],
                          ascending=False)
df_spark.show()

La base de données contient 14892 observations et 8 variables.
+-------------------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|               Date|Stock|       Open|       High|        Low|      Close|  Adj Close|  Volume|
+-------------------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|2022-01-11 00:00:00| TSLA|1053.670044|1075.849976|1038.819946|1064.400024|1064.400024|21937100|
|2022-01-10 00:00:00| TSLA|     1000.0|1059.099976|      980.0|1058.119995|1058.119995|30605000|
|2022-01-07 00:00:00| TSLA|1080.369995|1080.930054|     1010.0|1026.959961|1026.959961|27919000|
|2022-01-06 00:00:00| TSLA|     1077.0|     1088.0|     1020.5|1064.699951|1064.699951|30112200|
|2022-01-05 00:00:00| TSLA|1146.650024|1170.339966| 1081.01001|1088.119995|1088.119995|26706600|
|2022-01-04 00:00:00| TSLA|1189.550049|     1208.0|1123.050049|1149.589966|1149.589966|33416100|
|2022-01-03 00:00:00| TSLA|    1147.75|1201.069946|1136.040039|1

In [0]:
#On formatte la date pour enlever les heures et minutes
df_p = pyspark.sql.functions.split(df_spark['Date'], '\s')
df_spark= df_spark.withColumn('Date',df_p.getItem(0))
df_spark.show()

+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|      Date|Stock|       Open|       High|        Low|      Close|  Adj Close|  Volume|
+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+
|2022-01-11| TSLA|1053.670044|1075.849976|1038.819946|1064.400024|1064.400024|21937100|
|2022-01-10| TSLA|     1000.0|1059.099976|      980.0|1058.119995|1058.119995|30605000|
|2022-01-07| TSLA|1080.369995|1080.930054|     1010.0|1026.959961|1026.959961|27919000|
|2022-01-06| TSLA|     1077.0|     1088.0|     1020.5|1064.699951|1064.699951|30112200|
|2022-01-05| TSLA|1146.650024|1170.339966| 1081.01001|1088.119995|1088.119995|26706600|
|2022-01-04| TSLA|1189.550049|     1208.0|1123.050049|1149.589966|1149.589966|33416100|
|2022-01-03| TSLA|    1147.75|1201.069946|1136.040039|1199.780029|1199.780029|34643800|
|2021-12-31| TSLA|1073.439941|     1082.0|1054.589966|1056.780029|1056.780029|13528700|
|2021-12-30| TSLA|1061.329956|10

In [0]:
# On créée un index pour pouvoir déterminer un taux de variation des valeurs de fermeture
df_spark = df_spark.withColumn("id", monotonically_increasing_id())

In [0]:
my_window = Window.partitionBy().orderBy("id")

#on dréée d'abord 2 colonnes prev et diff qui serviront à calculer les taux
df_spark = df_spark.withColumn("prev_value", F.lag(df_spark.Close).over(my_window))

df_spark = df_spark.withColumn("diff", F.when(F.isnull(df_spark.Close - df_spark.prev_value), 0)
                              .otherwise(df_spark.Close - df_spark.prev_value))

df_spark = df_spark.withColumn('Variations %', F.col('diff') / F.col('Close'))
df_spark.show()

+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+---+-----------+-------------------+--------------------+
|      Date|Stock|       Open|       High|        Low|      Close|  Adj Close|  Volume| id| prev_value|               diff|        Variations %|
+----------+-----+-----------+-----------+-----------+-----------+-----------+--------+---+-----------+-------------------+--------------------+
|2022-01-11| TSLA|1053.670044|1075.849976|1038.819946|1064.400024|1064.400024|21937100|  0|       null|                0.0|                 0.0|
|2022-01-10| TSLA|     1000.0|1059.099976|      980.0|1058.119995|1058.119995|30605000|  1|1064.400024| -6.280029000000013|-0.00593508206032...|
|2022-01-07| TSLA|1080.369995|1080.930054|     1010.0|1026.959961|1026.959961|27919000|  2|1058.119995|-31.160033999999996|-0.03034201447314...|
|2022-01-06| TSLA|     1077.0|     1088.0|     1020.5|1064.699951|1064.699951|30112200|  3|1026.959961| 37.739990000000034|0.03544

In [0]:
to_round = ['Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'id',
 'prev_value',
 'diff',
 'Variations %']

for c in to_round:
    df_spark = df_spark.withColumn(c, round(c, 3))


df_spark.show()

+----------+-----+-------+-------+-------+-------+---------+--------+---+----------+------+------------+
|      Date|Stock|   Open|   High|    Low|  Close|Adj Close|  Volume| id|prev_value|  diff|Variations %|
+----------+-----+-------+-------+-------+-------+---------+--------+---+----------+------+------------+
|2022-01-11| TSLA|1053.67|1075.85|1038.82| 1064.4|   1064.4|21937100|  0|      null|   0.0|         0.0|
|2022-01-10| TSLA| 1000.0| 1059.1|  980.0|1058.12|  1058.12|30605000|  1|    1064.4| -6.28|      -0.006|
|2022-01-07| TSLA|1080.37|1080.93| 1010.0|1026.96|  1026.96|27919000|  2|   1058.12|-31.16|       -0.03|
|2022-01-06| TSLA| 1077.0| 1088.0| 1020.5| 1064.7|   1064.7|30112200|  3|   1026.96| 37.74|       0.035|
|2022-01-05| TSLA|1146.65|1170.34|1081.01|1088.12|  1088.12|26706600|  4|    1064.7| 23.42|       0.022|
|2022-01-04| TSLA|1189.55| 1208.0|1123.05|1149.59|  1149.59|33416100|  5|   1088.12| 61.47|       0.053|
|2022-01-03| TSLA|1147.75|1201.07|1136.04|1199.78|  119

In [0]:
df_spark = df_spark.select(
    ["Date",'Stock', 'Open', 'High', 'Close', 'Adj Close', 'Variations %']
                )
df_spark.show()

+----------+-----+-------+-------+-------+---------+------------+
|      Date|Stock|   Open|   High|  Close|Adj Close|Variations %|
+----------+-----+-------+-------+-------+---------+------------+
|2022-01-11| TSLA|1053.67|1075.85| 1064.4|   1064.4|         0.0|
|2022-01-10| TSLA| 1000.0| 1059.1|1058.12|  1058.12|      -0.006|
|2022-01-07| TSLA|1080.37|1080.93|1026.96|  1026.96|       -0.03|
|2022-01-06| TSLA| 1077.0| 1088.0| 1064.7|   1064.7|       0.035|
|2022-01-05| TSLA|1146.65|1170.34|1088.12|  1088.12|       0.022|
|2022-01-04| TSLA|1189.55| 1208.0|1149.59|  1149.59|       0.053|
|2022-01-03| TSLA|1147.75|1201.07|1199.78|  1199.78|       0.042|
|2021-12-31| TSLA|1073.44| 1082.0|1056.78|  1056.78|      -0.135|
|2021-12-30| TSLA|1061.33|1095.55|1070.34|  1070.34|       0.013|
|2021-12-29| TSLA|1098.64| 1104.0|1086.19|  1086.19|       0.015|
|2021-12-28| TSLA|1109.49| 1119.0|1088.47|  1088.47|       0.002|
|2021-12-27| TSLA|1073.67| 1117.0|1093.94|  1093.94|       0.005|
|2021-12-2

On note sur le haut des données une correction du cours de TSLA

## Statistiques des actions

Pour la suite du travail, on va se concentrer sur l'action AMZN.

Toutes les fonctions sont paramétrées et il suffit de changer le paramètre "action" pour analyser d'autres actions.

In [0]:
def statistiques(action, print_list = False):
    
    """
    Cette fonction permet de générer des statistiques d'une entreprise
    
    
    Parameters
    ----------
    action : str
        Nom du titre
    print_list : bool, , default=False
        Affiche ou non la liste des statistiques de base
    returns
    -----
        Des statistiques au format json
    """
    ## Stats
    url_stats = "https://finance.yahoo.com/quote/{}/key-statistics?p={}"
    print("--Statistiques de l'entreprise "+action+ "--")
    stock = action
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    response = requests.get(url_stats.format(stock, stock), headers = headers_)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    start = script_data.find("context") - 2
    json_data = json.loads(script_data[start:-12])
    #json_data['context'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["defaultKeyStatistics"].keys()
    mylist = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["defaultKeyStatistics"]

    if print_list == True:
        print(mylist)
        
    d = pd.DataFrame(json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["defaultKeyStatistics"])
    d = d[d.columns[d.isna().sum()<3]].replace(to_replace = np.NaN, value = "-")

    j = d.to_json(orient="columns")
    parsed = json.loads(j)
    print(json.dumps(parsed, indent=2))

    return 

statistiques("AMZN", print_list = False)

--Statistiques de l'entreprise AMZN--
{
  "enterpriseToRevenue": {
    "raw": 3.702,
    "fmt": "3.70",
    "longFmt": "-"
  },
  "profitMargins": {
    "raw": 0.057350002,
    "fmt": "5.74%",
    "longFmt": "-"
  },
  "enterpriseToEbitda": {
    "raw": 28.064,
    "fmt": "28.06",
    "longFmt": "-"
  },
  "52WeekChange": {
    "raw": 0.044647813,
    "fmt": "4.46%",
    "longFmt": "-"
  },
  "forwardEps": {
    "raw": 51.18,
    "fmt": "51.18",
    "longFmt": "-"
  },
  "sharesOutstanding": {
    "raw": 507148000,
    "fmt": "507.15M",
    "longFmt": "507,148,000"
  },
  "bookValue": {
    "raw": 237.799,
    "fmt": "237.80",
    "longFmt": "-"
  },
  "sharesShort": {
    "raw": 3599503,
    "fmt": "3.6M",
    "longFmt": "3,599,503"
  },
  "sharesPercentSharesOut": {
    "raw": 0.0070999996,
    "fmt": "0.71%",
    "longFmt": "-"
  },
  "lastFiscalYearEnd": {
    "raw": 1609372800,
    "fmt": "2020-12-31",
    "longFmt": "-"
  },
  "heldPercentInstitutions": {
    "raw": 0.59262,
    

## Statistiques financières

In [0]:
def financial_inf(action):

    """
    Cette fonction permet de générer des données financières d'une entreprise
    

    Parameters
    ----------
    action : str
        Nom du titre
    returns
    -----
        Les données de revenu, de la balance comptable et du cash flow au format annuel et trimestriel au format json
    """
    url_fin = "https://finance.yahoo.com/quote/{}/financials?p={}"
    stock = action
    headers_={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    response = requests.get(url_fin.format(stock, stock), headers = headers_)
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    #script_data[:500]
    #script_data[-500:]
    start = script_data.find("context") - 2

    json_data = json.loads(script_data[start:-12])
    #json_data['context'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

    annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["incomeStatementHistory"]["incomeStatementHistory"]
    #print(annual_is[0])
    #annual_is[0]["minorityInterest"]
    annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["cashflowStatementHistory"]["cashflowStatements"]
    annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["balanceSheetHistory"]["balanceSheetStatements"]

    quaterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["incomeStatementHistoryQuarterly"]["incomeStatementHistory"]
    quaterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["cashflowStatementHistoryQuarterly"]["cashflowStatements"]
    quaterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']["balanceSheetHistoryQuarterly"]["balanceSheetStatements"]

    return annual_is, annual_cf, annual_bs, quaterly_is, quaterly_cf, quaterly_bs

annual_is, annual_cf, annual_bs, quaterly_is, quaterly_cf, quaterly_bs = financial_inf("AMZN")

In [0]:
def income_stats():
  annual_is_stmts = []
  #annual
  for s in annual_is:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    annual_is_stmts.append(statement)
  
    #quaterly
  quaterly_is_stmts = []
  for s in quaterly_is:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    quaterly_is_stmts.append(statement)


  return annual_is_stmts, quaterly_is_stmts

annual_is_stmts, quaterly_is_stmts = income_stats()

In [0]:
annual_is_stmts

Out[48]: [{'researchDevelopment': 42740000000,
  'incomeBeforeTax': 24194000000,
  'netIncome': 21331000000,
  'sellingGeneralAdministrative': 87193000000,
  'grossProfit': 152757000000,
  'ebit': 22899000000,
  'endDate': 1609372800,
  'operatingIncome': 22899000000,
  'otherOperatingExpenses': -75000000,
  'interestExpense': -1647000000,
  'incomeTaxExpense': 2863000000,
  'totalRevenue': 386064000000,
  'totalOperatingExpenses': 363165000000,
  'costOfRevenue': 233307000000,
  'totalOtherIncomeExpenseNet': 1295000000,
  'netIncomeFromContinuingOps': 21331000000,
  'netIncomeApplicableToCommonShares': 21331000000},
 {'researchDevelopment': 35931000000,
  'incomeBeforeTax': 13962000000,
  'netIncome': 11588000000,
  'sellingGeneralAdministrative': 64313000000,
  'grossProfit': 114986000000,
  'ebit': 14404000000,
  'endDate': 1577750400,
  'operatingIncome': 14404000000,
  'otherOperatingExpenses': 338000000,
  'interestExpense': -1600000000,
  'incomeTaxExpense': 2374000000,
  'total

In [0]:
pd.DataFrame(quaterly_is_stmts)

,researchDevelopment,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
0,12097000000,4311000000,3156000000,30944000000,47882000000,4852000000,1632960000,4852000000,-11000000,-493000000,1155000000,110812000000,105960000000,62930000000,-541000000,3156000000,3156000000
1,11794000000,8646000000,7778000000,29397000000,48904000000,7702000000,1625011200,7702000000,11000000,-435000000,868000000,113080000000,105378000000,64176000000,944000000,7778000000,7778000000
2,10584000000,10263000000,8107000000,26628000000,46115000000,8865000000,1617148800,8865000000,38000000,-399000000,2156000000,108518000000,99653000000,62403000000,1398000000,8107000000,8107000000
3,16466000000,7787000000,7222000000,23428000000,46271000000,6873000000,1609372800,6873000000,-496000000,-414000000,565000000,125555000000,118682000000,79284000000,914000000,7222000000,7222000000


In [0]:
pd.DataFrame(annual_is_stmts)

,researchDevelopment,incomeBeforeTax,netIncome,sellingGeneralAdministrative,grossProfit,ebit,endDate,operatingIncome,otherOperatingExpenses,interestExpense,incomeTaxExpense,totalRevenue,totalOperatingExpenses,costOfRevenue,totalOtherIncomeExpenseNet,netIncomeFromContinuingOps,netIncomeApplicableToCommonShares
0,42740000000,24194000000,21331000000,87193000000,152757000000,22899000000,1609372800,22899000000,-75000000,-1647000000,2863000000,386064000000,363165000000,233307000000,1295000000,21331000000,21331000000
1,35931000000,13962000000,11588000000,64313000000,114986000000,14404000000,1577750400,14404000000,338000000,-1600000000,2374000000,280522000000,266118000000,165536000000,-442000000,11588000000,11588000000
2,28837000000,11270000000,10073000000,52177000000,93731000000,12421000000,1546214400,12421000000,296000000,-1417000000,1197000000,232887000000,220466000000,139156000000,-1151000000,10073000000,10073000000
3,22620000000,3802000000,3033000000,38992000000,65932000000,4106000000,1514678400,4106000000,214000000,-848000000,769000000,177866000000,173760000000,111934000000,-304000000,3033000000,3033000000


In [0]:
def clash_flow():
  annual_cf_stmts = []
  #annual
  for s in annual_cf:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    annual_cf_stmts.append(statement)

    #quaterly
  quaterly_cf_stmts = []
  for s in quaterly_cf:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    quaterly_cf_stmts.append(statement)


    return annual_cf_stmts, quaterly_cf_stmts
    
annual_cf_stmts, quaterly_cf_stmts = clash_flow()
annual_cf_stmts

Out[51]: [{'investments': -22242000000,
  'changeToLiabilities': 18745000000,
  'totalCashflowsFromInvestingActivities': -59611000000,
  'netBorrowings': -1104000000,
  'totalCashFromFinancingActivities': -1104000000,
  'changeToOperatingActivities': 5754000000,
  'netIncome': 21331000000,
  'changeInCash': 5967000000,
  'endDate': 1609372800,
  'effectOfExchangeRate': 618000000,
  'totalCashFromOperatingActivities': 66064000000,
  'depreciation': 25251000000,
  'changeToInventory': -2849000000,
  'changeToAccountReceivables': -8169000000,
  'changeToNetincome': 6001000000,
  'capitalExpenditures': -40140000000},
 {'investments': -9131000000,
  'changeToLiabilities': 9904000000,
  'totalCashflowsFromInvestingActivities': -24281000000,
  'netBorrowings': -10066000000,
  'totalCashFromFinancingActivities': -10066000000,
  'changeToOperatingActivities': -1383000000,
  'netIncome': 11588000000,
  'changeInCash': 4237000000,
  'endDate': 1577750400,
  'effectOfExchangeRate': 70000000,
  'to

In [0]:
def balance_sheet():
  annual_bs_stmts = []
  #annual
  for s in annual_bs:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    annual_bs_stmts.append(statement)

    #quaterly
  quaterly_bs_stmts = []
  for s in quaterly_bs:
    statement  = {}
    for key, val in s.items():
      try:
        statement[key] = val['raw']
      except TypeError:
        continue
      except KeyError:
        continue
    quaterly_bs_stmts.append(statement)


    return annual_bs_stmts, quaterly_bs_stmts
    
annual_bs_stmts, quaterly_bs_stmts = balance_sheet()
annual_bs_stmts

Out[52]: [{'intangibleAssets': 4981000000,
  'capitalSurplus': 42865000000,
  'totalLiab': 227791000000,
  'totalStockholderEquity': 93404000000,
  'otherCurrentLiab': 15267000000,
  'totalAssets': 321195000000,
  'endDate': 1609372800,
  'commonStock': 5000000,
  'otherCurrentAssets': 233000000,
  'retainedEarnings': 52551000000,
  'otherLiab': 17017000000,
  'goodWill': 15017000000,
  'treasuryStock': -2017000000,
  'otherAssets': 12097000000,
  'cash': 42122000000,
  'totalCurrentLiabilities': 126385000000,
  'shortLongTermDebt': 1155000000,
  'otherStockholderEquity': -180000000,
  'propertyPlantEquipment': 150667000000,
  'totalCurrentAssets': 132733000000,
  'longTermInvestments': 5700000000,
  'netTangibleAssets': 73406000000,
  'shortTermInvestments': 42274000000,
  'netReceivables': 24309000000,
  'longTermDebt': 31816000000,
  'inventory': 23795000000,
  'accountsPayable': 72539000000},
 {'intangibleAssets': 4049000000,
  'capitalSurplus': 33658000000,
  'totalLiab': 16318800

##Modélisations avec données historiques de AMAZON

In [0]:
df, url = WebScrapStock('AMZN', days_back=360*15).get_stock()
dff = df.copy()

stock = re.findall(r'[A-Z]+', url)[0]
period1 = re.findall(r'\d{5,}', url)[0]
period2 = re.findall(r'\d{5,}', url)[1]

def lien(stock, period1, period2):
    lien = "https://finance.yahoo.com/quote/"+stock+"/history?period1="+period1+"&period2="+period2+"&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
    return lien
lien = lien(stock, period1, period2)
print("Vous pouvez voir les données çà cette adresse : ", lien)

Vous pouvez voir les données çà cette adresse :  https://finance.yahoo.com/quote/AMZN/history?period1=1175402095&period2=1641962095&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true


In [0]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3723.000000,3723.000000,3723.000000,3723.000000,3723.000000,3.723000e+03
mean,886.492313,895.864909,876.117615,886.259412,886.259412,5.425956e+06
std,1020.606805,1031.342780,1008.587733,1019.923703,1019.923703,4.448423e+06
min,35.290001,39.000000,34.680000,35.029999,35.029999,8.813000e+05
25%,170.139999,172.069999,167.910004,170.110001,170.110001,3.009650e+06
50%,346.299988,351.049988,342.500000,346.380005,346.380005,4.299200e+06
75%,1548.344971,1571.184998,1520.410034,1544.925049,1544.925049,6.509000e+06
max,3744.000000,3773.080078,3696.790039,3731.409912,3731.409912,1.043292e+08


In [0]:
df.sort_values("Date", ascending=False)

,Date,Stock,Open,High,Low,Close,Adj Close,Volume
3722,2022-01-11,AMZN,3230.000000,3327.000000,3214.030029,3307.239990,3307.239990,3131500
3721,2022-01-10,AMZN,3211.709961,3233.229980,3126.090088,3229.719971,3229.719971,4389900
3720,2022-01-07,AMZN,3276.780029,3304.870117,3240.620117,3251.080078,3251.080078,2329300
3719,2022-01-06,AMZN,3269.010010,3296.000000,3238.739990,3265.080078,3265.080078,2597900
3718,2022-01-05,AMZN,3337.659912,3342.530029,3287.139893,3287.139893,3287.139893,3215100
...,...,...,...,...,...,...,...,...
4,2007-04-09,AMZN,41.720001,42.139999,41.610001,41.660000,41.660000,4182700
3,2007-04-05,AMZN,41.570000,41.759998,41.439999,41.680000,41.680000,3288400
2,2007-04-04,AMZN,41.220001,41.549999,40.919998,41.529999,41.529999,4064700
1,2007-04-03,AMZN,40.419998,41.380001,40.400002,41.189999,41.189999,5968200


In [0]:
df.shape

Out[56]: (3723, 8)

In [0]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(
x = df4["Date"],
y = df4["Close"],
mode = "lines",
name = "Cours de fermeture AMZN ",
marker = dict(color = 'black'))


fig.add_trace(trace1, secondary_y=False)


fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='Red', rangeslider_visible=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='black');
data = [trace1]

layout = dict(autosize=False,
                  width=1000,
                  height=550,
                  title = "Évolution du cours de l'action " +str(df.iloc[0, 1])+ " depuis " + str(dff.iloc[0, 0])[:10],
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de l'action",ticklen= 5, zeroline= True, showline=True, showgrid=False))
fig = dict(data = data, layout = layout)
#fig.update_xaxes()
iplot(fig)


In [0]:
df.set_index('Date', inplace=True)

Acheter des actions AMZN semble être une bonne idée

In [0]:
def preprocessing():
    
      """
        Cette fonction permet de transformer les données au format voulu

        returns
        -----
        Les inputs du modèle 
      """

      #X = df.drop(["Close", ])
      data = df.filter(["Close"])
      data_arr = data.values
      train_len = math.ceil(len(data_arr)*0.8)
      #print("Nombre d'observations des données d'entrainement", train_len)
      scaler = MinMaxScaler(feature_range=(0,1))
      data_scaled = scaler.fit_transform(data_arr)
      train = data_scaled[0:train_len, :]

      X_train = []
      y_train = []

      for i in range(60, len(train)):
        X_train.append(train[ i - 60 : i, 0])
        y_train.append(train[i, 0])

      #Conversion en array
      X_train, y_train = np.array(X_train), np.array(y_train)

      #Train set
      #reshape
      #On veut des 3D or on a des 2D
      print("--Train--")
      print(X_train.shape, "avant reshape")
      X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
      print(X_train.shape, "après reshape")

      #Test set

      test = data_scaled[train_len - 60 : , : ]
      X_test = []
      y_test = data_arr[train_len:, :]

      for i in range(60, len(test)):
        X_test.append(test[i - 60 : i, 0])

      #Conversion en array puis en 3D
      X_test = np.array(X_test)
      print("--Test--")
      print(X_test.shape, "avant reshape")
      X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
      print(X_test.shape, "après reshape")

      return X_train, y_train, X_test, y_test, train_len, scaler, data

X_train, y_train, X_test, y_test, train_len, scaler, data = preprocessing()

--Train--
(2919, 60) avant reshape
(2919, 60, 1) après reshape
--Test--
(744, 60) avant reshape
(744, 60, 1) après reshape


In [0]:
def modelisation():
      #LSTM ARN
    """
        Cette fonction entraine un modèle RNN afin de faire de la simulation in-sample des données du titre choisi 

        returns
        -----
        Les prédictions et le modèle entrainé
    """


    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape = (X_train.shape[1], 1)))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(100))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=1, epochs = 1)

    #Predictions
    y_pred = model.predict(X_test)
    y_pred = scaler.inverse_transform(y_pred)

    #metrics
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    print("RMSE du modèle", rmse)

    return y_pred, model

y_pred, model = modelisation()

2919/2919 [==============================] - 147s 49ms/step - loss: 5.6574e-04
RMSE du modèle 326.9491726332778


In [0]:
tr = data[:train_len]
val = data[train_len:]
val["Pred"] = y_pred
val = val.reset_index()
tr = tr.reset_index()

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

trace1 = go.Scatter(
x = tr["Date"],
y = tr["Close"],
mode = "lines",
name = "Données de d'entrainement",
marker = dict(color = 'black'))

trace2 = go.Scatter(
x = val["Date"],
y = val["Close"],
mode = "lines",
name = "Données de test",
marker = dict(color = 'blue'))

trace3 = go.Scatter(
x = val["Date"],
y = val["Pred"],
mode = "lines",
name = "données prédites",
marker = dict(color = '#CB7272'))
fig2.add_trace(trace2, secondary_y=False)
fig2.add_trace(trace3, secondary_y=False)

fig2.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='Red', rangeslider_visible=True)
fig2.update_yaxes( showline=True, linewidth=2, linecolor='black', gridcolor='black');

data = [trace1, trace2, trace3]
data1 = [trace2, trace3]
layout = dict(autosize=False,
                  width=1200,
                  height=550,
                  title = "Évolution du cours de l'action "+ str(dff.iloc[0, 1])+ " depuis " + str(dff.iloc[0, 0])[:10],
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de l'action",ticklen= 5, zeroline= True, showline=True, showgrid=False))

layout1 = dict(autosize=False,
                  width=1200,
                  height=550,
                  title = "Zoom sur la période de test",
xaxis= dict(title= 'Date', showgrid=False,showline=True),
yaxis= dict(title= "Cours de l'action",ticklen= 5, zeroline= True, showline=True, showgrid=False))
fig2 = dict(data = data, layout = layout)
fig3 = dict(data = data1, layout = layout1)
iplot(fig2)
iplot(fig3)


In [0]:
def predict_next_val():
  
    """
        Cette fonction permet de prédire la valeu du titre le prochain jour de cotation

        returns
        -----
        None
    """
    df1_close = df.filter(["Close"])

    last_60 = df1_close[-60:].values
    last_60_scl = scaler.transform(last_60)

    x_test = []
    x_test.append(last_60_scl)
    x_test = np.array(x_test)

    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    pred = model.predict(x_test)
    pred = scaler.inverse_transform(pred)

    print("La valeur de l'action pour le prochain jour de cotation est",pred[0][0], "$ en sachant qu'il est côté actuellement %.3f" %dff.iloc[-1, 5], "$")

    return 
predict_next_val()

La valeur de l'action pour le prochain jour de cotation est 2877.0051 $ en sachant qu'il est côté actuellement 3307.240 $
